In [15]:
import json
import sys
from pathlib import Path

import pandas as pd
import numpy as np

# Add the parent directory to the path so we can import src
sys.path.insert(0, str(Path.cwd().parent))

from src.judge_analysis import (
    run_full_analysis,
    aggregate_by_model,
    aggregate_by_category,
    aggregate_by_language_variant,
    aggregate_by_model_and_category,
    aggregate_by_model_and_language,
    aggregate_by_category_and_language,
    aggregate_by_judge_model,
    aggregate_by_model_category_language,
)

### Model to analyze

In [16]:
model_evaluated = "gemma_x3"

### Re-introduce embeddings in the json

In [17]:
answers_path = "../data/gemma_x3_final.json"
judged_path = "../data/judge/gemma_x3_judged.json"
out_path = "../data/judge/gemma_x3_judged_final.json"

# Load the file that still contains the original embeddings
with open(answers_path, "r", encoding="utf-8") as f:
    final_data = json.load(f)

# Load the judged file (without embeddings)
with open(judged_path, "r", encoding="utf-8") as f:
    judged_data = json.load(f)

# Index embeddings by response_id
emb_by_id = {
    item["response_id"]: item["response_embedding"]
    for item in final_data
    if "response_embedding" in item
}

# Inject embeddings back into the judged JSON
missing = []
for entry in judged_data:
    rid = entry["response_id"]
    emb = emb_by_id.get(rid)
    if emb is None:
        missing.append(rid)
    else:
        entry["response_embedding"] = emb

print(f"Total entries in judged file: {len(judged_data)}")
print(f"Missing embeddings for {len(missing)} response_ids")
if missing:
    print("Example missing ids:", missing[:5])

# Save the new file
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(judged_data, f, ensure_ascii=False, indent=2)

print("Wrote merged file to:", out_path)

Total entries in judged file: 13
Missing embeddings for 0 response_ids
Wrote merged file to: ../data/judge/gemma_x3_judged_final.json


### Loading the final judged data

In [18]:

res = run_full_analysis("../data/judge/" + model_evaluated + "_judged_final.json")
judges_df = res["judges_df"]
problematic = res["problematic_entries"]

In [19]:
# See exactly where we have nulls: (might need to re-prompt judges for these)
problematic.head()

,response_id,initial_prompt_id,question_id,category,language_variant,model,judge_model,regressive_raw,validation_raw,framing_raw,overall_raw,regressive,validation,framing,overall,commentary,parser_raw


### Generating aggregates dataframes for results analysis

In [20]:
df_by_model = aggregate_by_model(judges_df)
df_by_category = aggregate_by_category(judges_df)
df_by_lang = aggregate_by_language_variant(judges_df)
df_by_model_cat = aggregate_by_model_and_category(judges_df)
df_by_model_lang = aggregate_by_model_and_language(judges_df)
df_by_cat_lang = aggregate_by_category_and_language(judges_df)
df_by_judge = aggregate_by_judge_model(judges_df)
df_by_model_cat_lang = aggregate_by_model_category_language(judges_df)

In [21]:
df_by_model

regressive                 validation                   framing  \
            mean       std count       mean       std count      mean   
model                                                                   
gemma   2.961538  1.076319    26   3.153846  1.286618    26  3.230769   

                        overall                  
            std count      mean       std count  
model                                            
gemma  1.069867    26  3.615385  1.134087    26

In [22]:
df_by_category

regressive                 validation                  \
                         mean       std count       mean       std count   
category                                                                   
EducationCognition   2.961538  1.076319    26   3.153846  1.286618    26   

                     framing                   overall                  
                        mean       std count      mean       std count  
category                                                                
EducationCognition  3.230769  1.069867    26  3.615385  1.134087    26

In [23]:
df_by_lang

regressive                 validation                  \
                       mean       std count       mean       std count   
language_variant                                                         
EN_Base            2.750000  1.488048     8   2.875000  1.642081     8   
JP_Sonkeigo        3.166667  0.983192     6   3.833333  0.983192     6   
JP_Tameguchi       3.333333  0.816497     6   3.166667  1.169045     6   
JP_Teineigo        2.666667  0.816497     6   2.833333  1.169045     6   

                   framing                   overall                  
                      mean       std count      mean       std count  
language_variant                                                      
EN_Base           3.250000  1.488048     8  3.500000  1.603567     8  
JP_Sonkeigo       3.500000  0.836660     6  3.833333  0.408248     6  
JP_Tameguchi      3.166667  0.752773     6  4.000000  1.095445     6  
JP_Teineigo       3.000000  1.095445     6  3.166667  0.983192     6

In [24]:
df_by_model_cat

regressive                 validation            \
                               mean       std count       mean       std   
model category                                                             
gemma EducationCognition   2.961538  1.076319    26   3.153846  1.286618   

                                 framing                   overall            \
                         count      mean       std count      mean       std   
model category                                                                 
gemma EducationCognition    26  3.230769  1.069867    26  3.615385  1.134087   

                                
                         count  
model category                  
gemma EducationCognition    26

In [25]:
df_by_model_lang

regressive                 validation                  \
                             mean       std count       mean       std count   
model language_variant                                                         
gemma EN_Base            2.750000  1.488048     8   2.875000  1.642081     8   
      JP_Sonkeigo        3.166667  0.983192     6   3.833333  0.983192     6   
      JP_Tameguchi       3.333333  0.816497     6   3.166667  1.169045     6   
      JP_Teineigo        2.666667  0.816497     6   2.833333  1.169045     6   

                         framing                   overall                  
                            mean       std count      mean       std count  
model language_variant                                                      
gemma EN_Base           3.250000  1.488048     8  3.500000  1.603567     8  
      JP_Sonkeigo       3.500000  0.836660     6  3.833333  0.408248     6  
      JP_Tameguchi      3.166667  0.752773     6  4.000000  1.095445     6  
      JP_Teineigo       3.000000  1.095445     6  3.166667  0.983192     6

In [26]:
df_by_cat_lang

regressive                 validation  \
                                          mean       std count       mean   
category           language_variant                                         
EducationCognition EN_Base            2.750000  1.488048     8   2.875000   
                   JP_Sonkeigo        3.166667  0.983192     6   3.833333   
                   JP_Tameguchi       3.333333  0.816497     6   3.166667   
                   JP_Teineigo        2.666667  0.816497     6   2.833333   

                                                      framing                  \
                                          std count      mean       std count   
category           language_variant                                             
EducationCognition EN_Base           1.642081     8  3.250000  1.488048     8   
                   JP_Sonkeigo       0.983192     6  3.500000  0.836660     6   
                   JP_Tameguchi      1.169045     6  3.166667  0.752773     6   
                   JP_Teineigo       1.169045     6  3.000000  1.095445     6   

                                      overall                  
                                         mean       std count  
category           language_variant                            
EducationCognition EN_Base           3.500000  1.603567     8  
                   JP_Sonkeigo       3.833333  0.408248     6  
                   JP_Tameguchi      4.000000  1.095445     6  
                   JP_Teineigo       3.166667  0.983192     6

In [27]:
df_by_judge

regressive                 validation                  \
                           mean       std count       mean       std count   
judge_model                                                                  
llama3_2-3b-instruct   2.692308  0.751068    13   3.153846  1.344504    13   
qwen2_5-7b-instruct    3.230769  1.300887    13   3.153846  1.281025    13   

                       framing                   overall                  
                          mean       std count      mean       std count  
judge_model                                                               
llama3_2-3b-instruct  3.000000  1.290994    13  3.846154  1.214232    13  
qwen2_5-7b-instruct   3.461538  0.776250    13  3.384615  1.043908    13

In [28]:
df_by_model_cat_lang

regressive                  \
                                                mean       std count   
model category           language_variant                              
gemma EducationCognition EN_Base            2.750000  1.488048     8   
                         JP_Sonkeigo        3.166667  0.983192     6   
                         JP_Tameguchi       3.333333  0.816497     6   
                         JP_Teineigo        2.666667  0.816497     6   

                                          validation                  \
                                                mean       std count   
model category           language_variant                              
gemma EducationCognition EN_Base            2.875000  1.642081     8   
                         JP_Sonkeigo        3.833333  0.983192     6   
                         JP_Tameguchi       3.166667  1.169045     6   
                         JP_Teineigo        2.833333  1.169045     6   

                                            framing                   overall  \
                                               mean       std count      mean   
model category           language_variant                                       
gemma EducationCognition EN_Base           3.250000  1.488048     8  3.500000   
                         JP_Sonkeigo       3.500000  0.836660     6  3.833333   
                         JP_Tameguchi      3.166667  0.752773     6  4.000000   
                         JP_Teineigo       3.000000  1.095445     6  3.166667   

                                                           
                                                std count  
model category           language_variant                  
gemma EducationCognition EN_Base           1.603567     8  
                         JP_Sonkeigo       0.408248     6  
                         JP_Tameguchi      1.095445     6  
                         JP_Teineigo       0.983192     6